In [1]:
import threading
import time

import pandas as pd

import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from lib.trade_app import TradeApp
import lib.helper as helper

In [2]:
app = TradeApp()


In [3]:
"""Function that run the entire trading app """
app = TradeApp()
app.connect(host='127.0.0.1', port=7497,
                clientId=23)  # port 4002 for ib gateway paper trading/7497 for TWS paper trading
con_thread = threading.Thread(target=app.run, daemon=True)
con_thread.start()

tickers = ["FB", "AMZN", "INTC", "MSFT", "AAPL", "GOOG", "CSCO", "CMCSA", "ADBE", "NVDA",
               "NFLX", "PYPL", "AMGN", "AVGO", "TXN", "CHTR", "QCOM", "GILD", "FISV", "BKNG",
               "INTU", "ADP", "CME", "TMUS", "MU"]

capital = 300_000



ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:hfarm
ERROR -1 2104 Market data farm connection is OK:jfarm
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:eufarmnj
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil


NextValidId: 1


In [4]:
app.reqPositions()
time.sleep(2)
pos_df = app.pos_df
pos_df.drop_duplicates(inplace=True, ignore_index=True)  # position callback tends to give duplicate values
app.reqOpenOrders()
time.sleep(2)

Latest position data extracted 2
openOrderEnd.


In [5]:
for ticker in tickers:
    print("starting passthrough for.....",ticker)
    app.req_hist_data(tickers.index(ticker), helper.contract(ticker),'1 M', '15 mins')
    time.sleep(5)

starting passthrough for..... FB
0,FB,STK,,0,,,ISLAND,,USD,,,False,,,,combo:
starting passthrough for..... AMZN
0,AMZN,STK,,0,,,ISLAND,,USD,,,False,,,,combo:
starting passthrough for..... INTC
0,INTC,STK,,0,,,ISLAND,,USD,,,False,,,,combo:
starting passthrough for..... MSFT
0,MSFT,STK,,0,,,ISLAND,,USD,,,False,,,,combo:
starting passthrough for..... AAPL
0,AAPL,STK,,0,,,ISLAND,,USD,,,False,,,,combo:
starting passthrough for..... GOOG
0,GOOG,STK,,0,,,ISLAND,,USD,,,False,,,,combo:
starting passthrough for..... CSCO
0,CSCO,STK,,0,,,ISLAND,,USD,,,False,,,,combo:
starting passthrough for..... CMCSA
0,CMCSA,STK,,0,,,ISLAND,,USD,,,False,,,,combo:
starting passthrough for..... ADBE
0,ADBE,STK,,0,,,ISLAND,,USD,,,False,,,,combo:
starting passthrough for..... NVDA
0,NVDA,STK,,0,,,ISLAND,,USD,,,False,,,,combo:
starting passthrough for..... NFLX
0,NFLX,STK,,0,,,ISLAND,,USD,,,False,,,,combo:
starting passthrough for..... PYPL
0,PYPL,STK,,0,,,ISLAND,,USD,,,False,,,,combo:
starting passthrough for..... 

ERROR 0 200 No security definition has been found for the request
ERROR -1 2106 HMDS data farm connection is OK:ushmds.nj


In [9]:
app.hist_data[3]

[{'Date': '20230103 09:30:00 US/Eastern',
  'Open': 243.14,
  'High': 245.75,
  'Low': 241.72,
  'Close': 244.51,
  'Volume': Decimal('746458')},
 {'Date': '20230103 09:45:00 US/Eastern',
  'Open': 244.53,
  'High': 244.69,
  'Low': 241.25,
  'Close': 241.27,
  'Volume': Decimal('224056')},
 {'Date': '20230103 10:00:00 US/Eastern',
  'Open': 241.3,
  'High': 241.81,
  'Low': 240.2,
  'Close': 240.25,
  'Volume': Decimal('159551')},
 {'Date': '20230103 10:15:00 US/Eastern',
  'Open': 240.31,
  'High': 240.57,
  'Low': 239.35,
  'Close': 239.79,
  'Volume': Decimal('124856')},
 {'Date': '20230103 10:30:00 US/Eastern',
  'Open': 239.78,
  'High': 239.78,
  'Low': 238.64,
  'Close': 238.8,
  'Volume': Decimal('130231')},
 {'Date': '20230103 10:45:00 US/Eastern',
  'Open': 238.79,
  'High': 239.75,
  'Low': 238.31,
  'Close': 239.63,
  'Volume': Decimal('101149')},
 {'Date': '20230103 11:00:00 US/Eastern',
  'Open': 239.66,
  'High': 239.82,
  'Low': 237.5,
  'Close': 237.65,
  'Volume': De